In [31]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Comparison of mutational robustness of sRNA's vs. randomly generated RNA

## Imports

In [32]:
import os
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
import pandas as pd

from synbio_morpher.srv.io.manage.script_manager import script_preamble
from synbio_morpher.utils.common.setup import construct_circuit_from_cfg, prepare_config, expand_config
from synbio_morpher.utils.evolution.evolver import Evolver
from synbio_morpher.utils.circuit.agnostic_circuits.circuit_manager import CircuitModeller

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import jax
jax.config.update('jax_platform_name', 'cpu')

jax.devices()



[CpuDevice(id=0)]

In [33]:
fn = '../data/sRNA/merged_EcoCyc_RNAInter.csv'
data = pd.read_csv(fn)
data = data[data['Database'] == 'EcoCyc']
try:
    data = data.drop(columns=['Unnamed: 0'])
except:
    pass


In [34]:
circuit_fasta = os.path.join('data', 'merged.fasta')
circuit_fasta_d = {}
with open(circuit_fasta, 'w') as f:

    for n in list(data['Name'].unique())[:3]:
        circuit_fasta_d[n] = data[data['Name'] == n]['Sequence'].iloc[0]
        f.write('>' + n.replace(' ', '_') + '\n')
        f.write(circuit_fasta_d[n])
        f.write('\n')

In [39]:
nn = np.sum(np.arange(1, len(circuit_fasta_d)+1)) + len(circuit_fasta_d)
print('Expected number of species: ', nn)

Expected number of species:  9


# Config

In [40]:
config = {
    "experiment": {
        "purpose": "tests",
        "no_visualisations": False,
        "no_numerical": False,
        "debug_mode": False
    },
    "data_path": circuit_fasta,
    "system_type": "RNA",
    "include_prod_deg": False,
    "interaction_simulator": {
        "name": "IntaRNA",
        "postprocess": True
    },
    "mutations_args": {
        "algorithm": "all",
        "mutation_counts": 0,
        "mutation_nums_within_sequence": [1],
        "mutation_nums_per_position": 1,
        "concurrent_species_to_mutate": "single_species_at_a_time",
        "seed": 0
    },
    "signal": {
        "inputs": [data['Name'].iloc[0]],
        "outputs": [],
        "function_name": "step_function",
        "function_kwargs": {
            "impulse_center": 1,
            "impulse_halfwidth": 5,
            "target": 2
        }
    },
    "simulation": {
        "dt0": 0.1,
        "t0": 0,
        "t1": 100,
        "tmax": 2000,
        "solver": "diffrax",
        "use_batch_mutations": True,
        "interaction_factor": 1,
        "batch_size": 20000,
        "max_circuits": 60000,
        "device": "cpu",
        "threshold_steady_states": 0.05,
        "use_rate_scaling": True
    },
    "molecular_params_factor": 1,
    "molecular_params": {
        "avg_mRNA_per_cell": 100,
        "cell_doubling_time": 1200,
        "creation_rate": 2.35,
        "starting_copynumbers": 200,
        "degradation_rate": 0.01175,
        "association_binding_rate": 1000000
    }
}

# Simulate

In [41]:
data_writer = None
config, data_writer = script_preamble(config, data_writer)
config_file = expand_config(config=config)
config_file = prepare_config(config_file)
circuit = construct_circuit_from_cfg(prev_configs=None, config_file=config_file)

len(circuit.model.species)


9

In [38]:
modeller = CircuitModeller(result_writer=data_writer, config=config_file)
circuits = modeller.batch_circuits(
    circuits=[circuit],
    write_to_subsystem=True, batch_size=config_file['simulation'].get('batch_size', 100),
    methods={
        "compute_interactions": {},
        # "init_circuits": {'batch': True},
        # "simulate_signal_batch": {'ref_circuit': None,
        #                           'batch': True},
        "write_results": {'no_visualisations': config_file['experiment'].get('no_visualisations', True),
                          'no_numerical': config_file['experiment'].get('no_numerical', False)}
    })

ModuleNotFoundError: No module named 'synbio_morpher.srv.parameter_prediction.IntaRNA'

# Compare